In [1]:
import numpy as np
from cosmoTransitions import generic_potential_1
import matplotlib.pyplot as plt
import pandas as pd
from scipy import optimize
import scipy.integrate as integrate
import random
from scipy import interpolate, special
import seaborn as sns
from scipy import misc



####Some definitions##
v2 = 246.2**2
mh=125.09
v=246.2
alpha=1/137
sinthw=np.sqrt(0.223)
g1=np.sqrt(4*np.pi*alpha/(1-sinthw**2))
g=np.sqrt(4*np.pi*alpha)/sinthw
Mplanck=2.4*10**18
cs=1/3**0.5 ##Sound speed constant


####This code uses an interpoaltion function for the number of degrees of freedom as function of temperature
###Data is obtained from https://member.ipmu.jp/satoshi.shirai/EOS2018
data = np.loadtxt( 'satoshi_dof.dat' )
Temperature_d=(data.T)[0][900:3900]
dof_d=(data.T)[1][900:3900]
#f = interpolate.interp1d(Temperature_d, dof_d)###"""the function works from T=[10e-4,1000]"""
g_star = interpolate.interp1d(Temperature_d, dof_d, kind='cubic')


class model1(generic_potential_1.generic_potential):
    def init(self, ms = 50, theta = 0, muhs = 0, u = 100, mu3 = 0):
        self.Ndim = 2
        self.renormScaleSq = v2
        self.ms = ms
        self.theta = theta
        self.muhs = muhs
        self.u = u
        self.mu3 = mu3
        self.lamh = 1/(4*v2)*(mh**2+self.ms**2 + (mh**2 - ms**2)*np.cos(2*self.theta))
        #self.lams = 1/(2*self.u**2)*(mh**2*np.sin(self.theta)**2+self.ms**2*np.cos(self.theta)**2 + self.mu3*self.u + self.muhs*v**2/(2*self.u))
        self.lams = 1/(4*self.u**3)*(mh**2*self.u + ms**2*self.u + 2*self.u**2*self.mu3 + v**2*self.muhs - (mh**2-ms**2)*self.u*np.cos(2*self.theta))
        self.lammix = 1/(v*self.u)*(-(self.ms**2-mh**2)*np.sin(self.theta)*np.cos(self.theta) - self.muhs*v)
        self.muh2 = self.lamh*v2 + self.muhs*self.u + self.lammix/2*self.u**2
        self.mus2 = -self.mu3*self.u + self.lams*self.u**2 + self.muhs*v2/(2*self.u) + self.lammix/2*v2

    def forbidPhaseCrit(self, X):
        return any([np.array([X])[...,0] < -5.0])
        #return any([np.array([X])[...,0] < -5.0, np.array([X])[...,1] < -5.0])

    def V0(self, X):
        X = np.asanyarray(X)
        h, s = X[...,0], X[...,1]
        pot = -1/2*self.muh2*h**2 + 1/4*self.lamh*h**4 - 1/2*self.mus2*s**2 - 1/3*self.mu3*s**3 + 1/4*self.lams*s**4 + 1/2*self.muhs*h**2*s + 1/4*self.lammix*h**2*s**2
        return pot

    def boson_massSq(self, X, T):
        X = np.array(X)
        h, s = X[...,0], X[...,1]

       #####Scalar thermal masses, obtained from appendix of 1702.06124
        Pi_h = T**2*(g1**2/16 + 3*g**2/16 + self.lamh/2 + 1/4 + self.lammix/24)
        Pi_s= T**2*(self.lammix/6 + self.lams/4)

        ##Scalar mass matrix##
        a=3*h**2*self.lamh + s**2*self.lammix/2 - self.muh2 + s*self.muhs + Pi_h
        b=h**2*self.lammix/2 + 3*s**2*self.lams - 2*s*self.mu3 - self.mus2 + Pi_s
        cc=h*s*self.lammix  + h*self.muhs
        A=(a+b)/2
        B=1/2*np.sqrt((a-b)**2+4*cc**2)
        m1=A+B
        m2=A-B

        ####Gauge boson masses (Longitudinal)
        mWL = g**2*h**2/4 + 11/6*g**2*T**2
        ag=g**2*h**2/4 + 11/6*g**2*T**2
        bg=1/4*g1**2*h**2 + 11/6*g1**2*T**2
        ccg=-1/4*g1*g*h**2
        Ag=(ag+bg)/2
        Bg=1/2*np.sqrt((ag-bg)**2+4*ccg**2)
        mZL=Ag+Bg
        mPh=Ag-Bg


        M = np.array([m1,m2,g**2*h**2/4,h**2/4*(g**2+g1**2),mWL,mZL])
        if self.ms<mh:
            Mphys = np.array([mh**2,self.ms**2,g**2*v**2/4,v**2/4*(g**2+g1**2),g**2*v**2/4,v**2/4*(g**2+g1**2)])
        else:
            Mphys = np.array([self.ms**2,mh**2,g**2*v**2/4,v**2/4*(g**2+g1**2),g**2*v**2/4,v**2/4*(g**2+g1**2)])

        # At this point, we have an array of boson masses, but each entry might
        # be an array itself. This happens if the input X is an array of points.
        # The generic_potential class requires that the output of this function
        # have the different masses lie along the last axis, just like the
        # different fields lie along the last axis of X, so we need to reorder
        # the axes. The next line does this, and should probably be included in
        # all subclasses.
        M = np.rollaxis(M, 0, len(M.shape))
        Mphys = np.rollaxis(Mphys, 0, len(Mphys.shape))

        # The number of degrees of freedom for the masses. This should be a
        # one-dimensional array with the same number of entries as there are
        # masses.

        dof = np.array([1,1,4,2 , 2,1]) ##Longitudinal at the end


        # c is a constant for each particle used in the Coleman-Weinberg
        # potential using MS-bar renormalization. It equals 1.5 for all scalars
        # and the longitudinal polarizations of the gauge bosons, and 0.5 for
        # transverse gauge bosons.
        #c = np.array([1.5,1.5,1.5,1.5,1.5,1.5,1.5])
        c = np.array([1.5,1.5,1.5,1.5,1.5,1.5])

        return M, dof, c, Mphys

    def old_boson_massSq(self, X, T):
        X = np.array(X)
        h, s = X[...,0], X[...,1]


        #####Scalar thermal masses, obtained from appendix of 1702.06124
        Pi_h = T**2*(g1**2/16 + 3*g**2/16 + self.lamh/2 + 1/4 + self.lammix/24)
        Pi_s= T**2*(self.lammix/6 + self.lams/4)

        ##Scalar mass matrix##
        a=3*h**2*self.lamh + s**2*self.lammix/2 - self.muh2 + s*self.muhs + Pi_h
        b=h**2*self.lammix/2 + 3*s**2*self.lams - 2*s*self.mu3 - self.mus2 + Pi_s
        cc=h*s*self.lammix  + h*self.muhs
        A=(a+b)/2
        B=1/2*np.sqrt((a-b)**2+4*cc**2)
        m1=A+B
        m2=A-B

        ####Gauge boson masses
        mW = g**2*h**2/4 + 11/6*g**2*T**2
        ag=g**2*h**2/4 + 11/6*g**2*T**2
        bg=1/4*g1**2*h**2 + 11/6*g1**2*T**2
        ccg=-1/4*g1*g*h**2
        Ag=(ag+bg)/2
        Bg=1/2*np.sqrt((ag-bg)**2+4*ccg**2)
        mZ=Ag+Bg
        mPh=Ag-Bg


        M = np.array([m1,m2,mW,mZ])
        if self.ms<mh:
            Mphys = np.array([mh**2,self.ms**2,g**2*v**2/4,v**2/4*(g**2+g1**2)])
        else:
            Mphys = np.array([self.ms**2,mh**2,g**2*v**2/4,v**2/4*(g**2+g1**2)])

        # At this point, we have an array of boson masses, but each entry might
        # be an array itself. This happens if the input X is an array of points.
        # The generic_potential class requires that the output of this function
        # have the different masses lie along the last axis, just like the
        # different fields lie along the last axis of X, so we need to reorder
        # the axes. The next line does this, and should probably be included in
        # all subclasses.
        M = np.rollaxis(M, 0, len(M.shape))
        Mphys = np.rollaxis(Mphys, 0, len(Mphys.shape))

        # The number of degrees of freedom for the masses. This should be a
        # one-dimensional array with the same number of entries as there are
        # masses.

        dof = np.array([1,1,6,3])


        # c is a constant for each particle used in the Coleman-Weinberg
        # potential using MS-bar renormalization. It equals 1.5 for all scalars
        # and the longitudinal polarizations of the gauge bosons, and 0.5 for
        # transverse gauge bosons.
        #c = np.array([1.5,1.5,1.5,1.5,1.5,1.5,1.5])
        c = np.array([1.5,1.5,1.5,1.5])

        return M, dof, c, Mphys


    def fermion_massSq(self, X):
        X = np.array(X)
        h,s = X[...,0], X[...,1]
        mt=h**2/2
        M = np.array([mt])
        Mphys = np.array([v**2/2])

        # At this point, we have an array of boson masses, but each entry might
        # be an array itself. This happens if the input X is an array of points.
        # The generic_potential class requires that the output of this function
        # have the different masses lie along the last axis, just like the
        # different fields lie along the last axis of X, so we need to reorder
        # the axes. The next line does this, and should probably be included in
        # all subclasses.
        M = np.rollaxis(M, 0, len(M.shape))
        Mphys = np.rollaxis(Mphys, 0, len(Mphys.shape))

        dof = np.array([12])
        return M, dof, Mphys


    def approxZeroTMin(self):
        # There are generically two minima at zero temperature in this model,
        # and we want to include both of them.

        return [np.array([v,self.u])]

    def theory_consistent(self):
        perturbative_limit=4*np.pi
        perturbativity=self.lamh<=perturbative_limit and self.lams<=perturbative_limit and abs(self.lammix)<=perturbative_limit
        positivity=(self.lamh>0) and (self.lams>0) and (self.lammix>-2*(self.lamh*self.lams)**.5)
        if perturbativity and positivity:
            print("Model is theoretically consistent \n")
            return True
        else:
            print("Model is NOT theoretically consistent \n")
            return False


    def print_couplings(self):
        print("Potential parameters are given by \n ")
        print("mus2=",self.mus2, "muh2=",self.muh2,"lamh=",self.lamh,"lams=",self.lams,"lammix=",self.lammix,"\n")
        print("Model parameters are \n")
        print("ms=",self.ms,"theta=",self.theta,"muhs=",self.muhs,"u=",self.u,"mu3=",self.mu3,"\n")

    def isEWSB(self):
        """Method to find the deepest minima of the potential at T=0.
        Doesn't work for Z_2 symmetric potential!!!"""
        n=100
        X_EW=np.array([v,self.u])
        minima=[]
        if self.muhs==0 and self.mu3==0:
            print("Model has a Z2 symmetry in the potential \n")
            print("isEWSB=True \n")
            return True
        #------------
        X0=self.findMinimum([0,100],0)
        if self.Vtot(X0,0)<=self.Vtot(X_EW,0) and abs(abs(X0[0])-v)>10 and abs(self.Vtot(X0,0)-self.Vtot(X_EW,0))>1:
            print("Global minimum found at X=",X0,"\n")
            print("isEWSB=False \n")
            return False
        X0=self.findMinimum([0,-100],0)
        if m.Vtot(X0,0)<=m.Vtot(X_EW,0) and abs(abs(X0[0])-v)>10 and abs(self.Vtot(X0,0)-self.Vtot(X_EW,0))>1:
            print("Global minimum found at X=",X0,"\n")
            print("isEWSB=False \n")
            return False
         
        ###This loop search for a global minima randomly
        for i in range(n):
            x1=np.random.uniform(-100,4*self.Tmax)
            x2=np.random.uniform(-4*self.Tmax,4*self.Tmax)
            X0=self.findMinimum([x1,x2], T=0.0)
            if self.Vtot(X0,0)<=self.Vtot(X_EW,0) and abs(X0[0])-v>10 and abs(self.Vtot(X0,0)-self.Vtot(X_EW,0))>1e2:
                print("Global minimum found at X=",X0,"\n")
                print("isEWSB=False \n")
                return False
        print("isEWSB=True \n")
        return True




def alpha_GW(Tnuc,Drho):
    ####This code gives the parameter alpha relevant for stochastic GW spectrum
    ##AS APPEAR IN FORMULA (8.2) OF 1912.12634
    num_dof=g_star(Tnuc)
    radiationDensity=np.pi**2/30*num_dof*Tnuc**4
    latentHeat=Drho
    return latentHeat/radiationDensity


def trans_class(SymNR):
    """Classify the transition according to the following characteristics:
    ---------
    phi-sym: transition happens in the s-field direction
    phi-symNR: transition happens in the s-field direction (symmetry is not restored at T=1000)
    """
    SNR="sym"
    if SymNR==True:
        SNR="SNR"

    if dh>10 and ds>10:
        return "hs-"+SNR
    elif dh>10 and ds<1:
        return "h-"+SNR
    elif ds>10 and dh<1:
        return "s-"+SNR
    else:
        return "none"+SNR

def beta_GW(Tnuc,dS_TdT):
    ###This code defines the parameter beta relevant for stochastic GW spectrum
    beta=Tnuc*dS_TdT
    return beta

def S_profile(T,elem):
    """This function calculates the Euclidean action from a model m at temperature T
    after knowing its phase history. If more than one FOPT is found it uses "elem" from the list. 
    """
    profile=elem["instanton"].profile1D
    alpha_ode=2
    temp=T
    r, phi, dphi, phivector = profile.R, profile.Phi, profile.dPhi, elem["instanton"].Phi
    phi_meta=elem["high_vev"]
    # Find the area of an n-sphere (alpha=n):
    d = alpha_ode+1  # Number of dimensions in the integration
    area = r**alpha_ode * 2*np.pi**(d*.5)/special.gamma(d*.5) ##4 pi r^2 the surface of a sphere
    # And integrate the profile
    integrand = 0.5 * dphi**2 + m.Vtot(phivector,temp) - m.Vtot(phi_meta,temp)
    integrand *= area
    S = integrate.simps(integrand, r)
    # Find the bulk term in the bubble interior
    volume = r[0]**d * np.pi**(d*.5)/special.gamma(d*.5 + 1)
    S += volume * (m.Vtot(phivector[0],temp) - m.Vtot(phi_meta,temp))

    return S/T

   

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Compute the transition of model(s)

In [2]:
df=pd.read_csv("SCANS/BAU/Z2_breaking_sols_BAU.csv",index_col=[0]).sort_values("alpha_max").drop_duplicates()
df=df[df.Tnuc_0>df.dT_0]



#np.random.seed()
#modi=np.random.randint(0,len(df))
modi=-1
ms_val=df.iloc[modi]["ms"]
theta_val=df.iloc[modi]["theta"]
u_val=df.iloc[modi]["u"]
mu3_val=df.iloc[modi]["mu3"]
muhs_val=df.iloc[modi]["muhs"]

#m=model1(ms = ms_val*(1+np.random.uniform(-.05,0.05)), 
#         theta = theta_val*(1+np.random.uniform(-.05,0.05)),
#         muhs= muhs_val*(1+np.random.uniform(-.05,0.05)) ,
#         u = u_val*(1+np.random.uniform(-.05,0.05)),
#         mu3 = mu3_val*(1+np.random.uniform(-.05,0.05)))

m=model1(ms = ms_val, theta = theta_val,muhs= muhs_val ,u = u_val,mu3 = mu3_val)

m.print_couplings()
thbool=m.theory_consistent()
EWSBbool=m.isEWSB()

Pih=g1**2/16 + 3*g**2/16 + m.lamh/2 + 1/4 + m.lammix/24
Pis=m.lammix/6 + m.lams/4
lamh_tilde=m.lamh - m.lammix**2/4/m.lams
dict_out={'ms':m.ms,'theta':m.theta, 'u':m.u,"muhs":m.muhs,"mu3":m.mu3,
      "lamh":m.lamh,"lams":m.lams,"lammix":m.lammix,
      "muh2":m.muh2,"mus2":m.mus2,
      "Pih":Pih,"Pis":Pis,"lamh_tilde":lamh_tilde}
dict_out.update({ "th_bool":thbool and EWSBbool})
if thbool==True and EWSBbool==True:
    try:
        alltrans=m.findAllTransitions()
        index=0
        count_trans=0
        alpha_list=[]
        dT_list=[]
        trans_types=[]
        for elem in alltrans:
            if elem["trantype"]==1:
                count_trans+=1
                phi_stable=elem["low_vev"]
                phi_meta=elem["high_vev"]
                SymNR=np.sum(m.findMinimum([0,0],1000)**2)**0.5>10
                dh,ds=abs(phi_meta-phi_stable)
                trans_types.append(trans_class(SymNR))
                Tnuc=elem["Tnuc"]
                #Tc=elem["crit_trans"]["Tcrit"]
                dT=abs(m.phases[elem["high_phase"]].T[0]-m.phases[elem["low_phase"]].T[-1])
                dT_list.append(dT)
                Delta_rho=m.energyDensity(phi_meta,Tnuc,include_radiation=True)-m.energyDensity(phi_stable,Tnuc,include_radiation=True)
                alpha=alpha_GW(Tnuc,Delta_rho)
                dS_TdT_Tnuc=misc.derivative(S_profile, x0=Tnuc, dx=.01, n=1, args=(elem,),order=7)
                beta_Tnuc=beta_GW(Tnuc,dS_TdT_Tnuc)
                alpha_list.append(alpha)
                Delta_pressure=m.Vtot(phi_meta,Tnuc) -m.Vtot(phi_stable,Tnuc)
                vwformula=(Delta_pressure/Delta_rho)**0.5
                xi_Jouguet=((alpha*(2+3*alpha))**0.5+1)/(3**0.5*(1+alpha))
                v_calculable=vwformula<xi_Jouguet
                dict_out.update({ "h_low_"+str(index):phi_stable[0],"s_low_"+str(index):phi_stable[1],
                                 "h_high_"+str(index):phi_meta[0],"s_high_"+str(index):phi_meta[1],
                                 "Tnuc_"+str(index):Tnuc,"dT_"+str(index):dT,
                                 "alpha_"+str(index):alpha,"beta_"+str(index):beta_Tnuc,
                                "vwf_"+str(index):vwformula,"xi_J_"+str(index):xi_Jouguet,
                                 "v_calculable_"+str(index):vwformula<xi_Jouguet})
                index+=1
        relevant_index=alpha_list.index(max(alpha_list))
        dict_out.update({"num_FOPT":count_trans,"alpha_max":max(alpha_list),
                         "dT_max":dT_list[relevant_index],
                         "tran_type":trans_types[relevant_index]})

    except:
        print("error ocurred")
        

Potential parameters are given by 
 
mus2= 1849.4876993978833 muh2= -18879.96262937881 lamh= 0.19081251087831483 lams= 2.379244889964469 lammix= 1.1419815248757814 

Model parameters are 

ms= 382.3185715425311 theta= 0.2418149145930677 muhs= -291.0832744480213 u= 146.9614892167512 mu3= 164.11200997527743 

Model is theoretically consistent 

isEWSB=True 

Tracing phase starting at x = [246.19999665 146.9614892 ] ; t = 0.0
Tracing minimum up
traceMinimum t0 = 0
......................................................................................
Tracing phase starting at x = [7.35051433e-07 5.46023837e+01] ; t = 124.66297293612857
Tracing minimum down
traceMinimum t0 = 124.663
................................................................
Tracing minimum up
traceMinimum t0 = 124.663
...............................................................................................................................................
Tracing phase starting at x = [-4.69931457e-07  5.60953741e

In [3]:
dict_out

{'ms': 382.3185715425311,
 'theta': 0.2418149145930677,
 'u': 146.9614892167512,
 'muhs': -291.0832744480213,
 'mu3': 164.11200997527743,
 'lamh': 0.19081251087831483,
 'lams': 2.379244889964469,
 'lammix': 1.1419815248757814,
 'muh2': -18879.96262937881,
 'mus2': 1849.4876993978833,
 'Pih': 0.47749030088772515,
 'Pis': 0.7851414766370809,
 'lamh_tilde': 0.05378145024030184,
 'th_bool': True,
 'h_low_0': 226.66272765859014,
 's_low_0': 142.36821924030284,
 'h_high_0': -2.8733737285136375e-07,
 's_high_0': 55.86164730165244,
 'Tnuc_0': 90.32432742306239,
 'dT_0': 39.077207631200324,
 'alpha_0': 0.02820729284209725,
 'beta_0': 690.5116170903766,
 'vwf_0': 0.6999270531441955,
 'xi_J_0': 0.6976726456405894,
 'v_calculable_0': False,
 'num_FOPT': 1,
 'alpha_max': 0.02820729284209725,
 'dT_max': 39.077207631200324,
 'tran_type': 'hs-SNR'}

In [4]:
phases_old=m.phases
phases_old

{0: Phase(key=0, X=[[246.2 147], ..., [149.1 120.5]], T=[0, ..., 123.7], dXdT=[[-0 -0], ..., [-130.4 -41.75]],
 1: Phase(key=1, X=[[7.665e-06 56.05], ..., [0.0005187 44.41]], T=[84.58, ..., 1000], dXdT=[[-4.314e-05 -0.1349], ..., [0.0001258 -0.002377]],
 2: Phase(key=2, X=[[-1.38e-07 59.08], ..., [-4.24e-07 56.06]], T=[0, ..., 84.06], dXdT=[[-0 0], ..., [1.694e-07 -0.0005107]]}

In [5]:
def my_getPhases(m):
    myexps=[(-5,-3),(-5,-5),(-5,-4),(-3,-3)]
    for nord in myexps:
        print("doing",nord)
        try:
            m.getPhases(tracingArgs={"dtstart":10**(nord[0]), "tjump":10**(nord[1])})
            phases_out=m.phases
        except:
            phases_out={}
        finally:
            if len(phases_out)>1:
                break
    return phases_out

def find_nucleation(m):
    """Find min and max temperatures to search for nucleation. IT will be used by bisection method.
    Parameters
        ----------
        m: a model instance. In this case m=model1(kk=1/600**2) for example.
    Returns
        -------
        nuc_dict: a dictionary containing the relevant temperatures and phases indexes.
                It will be used by the other methods to find the nucleation and percolation parameters
    """
    if m.phases is None:
        try:
            #phases_dict=m.getPhases()
            #phases_dict=m.getPhases(tracingArgs={"dtstart":1e-3, "tjump":1e-3})
            #phases_dict=m.getPhases(tracingArgs={"dtstart":1e-5, "tjump":1e-4})
            phases_dict=my_getPhases(m)
        except:
            return {}
    else:
        phases_dict=m.phases
    if len(phases_dict)<=1:
        return {}
    from cosmoTransitions import transitionFinder as tf
    crit_temps=tf.findCriticalTemperatures(phases_dict, m.Vtot)
    Num_relevant_trans=0
    ###DETERMINE IF THERE COULD BE TWO-STEP FOPTs
    for elem in crit_temps:
        if elem["trantype"]==1 and abs(elem["low_vev"][0]-elem["high_vev"][0])>10 and abs(elem["low_vev"][1]-elem["high_vev"][1])>10:
            print("Tunneling is relevant from phase " + str(elem["high_phase"])+ " to " + str(elem["low_phase"])  )
            Tmax=elem["Tcrit"]
            Tmin=phases_dict[elem["high_phase"]].T[0]
            print("max temperature is", Tmax)
            print("min temperature is", Tmin)
            Num_relevant_trans+=1
            high_phase_key=elem["high_phase"]
            low_phase_key=elem["low_phase"]
        else:
            continue
    if Num_relevant_trans==0:
        dict_output={}
        return dict_output
    else:
        dict_output= {"Tmin":Tmin, "Tmax":Tmax, "high_phase": high_phase_key,"low_phase": low_phase_key}
    X0=m.phases[dict_output["high_phase"]].X[0]
    T0=m.phases[dict_output["high_phase"]].T[0]
    stable=not np.any(np.linalg.eig(m.d2V(X0,T0))[0]<=0)

    def findminT(T):
        """Function to find the minimum temperature at which the high_vev coexists.
        Written in form for optimization"""
        Xmin=m.findMinimum(X0,T)
        dx=np.sum((Xmin-X0)**2)**0.5
        stable=not np.any(np.linalg.eig(m.d2V(Xmin,T))[0]<=0)
        if stable and (dx<.1) == False or T<0:
            return 500
        else:
            return stable and (dx<.1) + T
    Tmin_opt=optimize.fminbound(findminT,0,T0)
    dict_output["Tmin"]=Tmin_opt
    return dict_output

In [6]:
df=pd.read_csv("SCANS/BAU/Z2_breaking_sols_BAU.csv",index_col=[0]).sort_values("alpha_max").drop_duplicates()
df=df[df.Tnuc_0>df.dT_0]

modi=-1
ms_val=df.iloc[modi]["ms"]
theta_val=df.iloc[modi]["theta"]
u_val=df.iloc[modi]["u"]
mu3_val=df.iloc[modi]["mu3"]
muhs_val=df.iloc[modi]["muhs"]

m=model1(ms = ms_val, theta = theta_val,muhs= muhs_val ,u = u_val,mu3 = mu3_val)
#nuc_dict=find_nucleation(m)

phases_dict=my_getPhases(m)

doing (-5, -3)
Tracing phase starting at x = [246.19999665 146.9614892 ] ; t = 0.0
Tracing minimum up
traceMinimum t0 = 0
..............................................................................................................................................................
Tracing phase starting at x = [1.50603685e-06 5.46024233e+01] ; t = 124.6620958031951
Tracing minimum down
traceMinimum t0 = 124.662
................................................................
Tracing minimum up
traceMinimum t0 = 124.662
...................................................................................................................................................
Tracing phase starting at x = [6.46116226e-10 5.60958940e+01] ; t = 83.57735883083726
Tracing minimum down
traceMinimum t0 = 83.5774
...........................................................................................
Tracing minimum up
traceMinimum t0 = 83.5774
..................


In [7]:
phases_dict

{0: Phase(key=0, X=[[246.2 147], ..., [149.1 120.5]], T=[0, ..., 123.7], dXdT=[[-0 -0], ..., [-132.7 -42.5]],
 1: Phase(key=1, X=[[3.084e-05 56.05], ..., [0.0003293 44.41]], T=[84.58, ..., 1000], dXdT=[[-0.0001757 -0.1349], ..., [0.0001306 -0.002188]],
 2: Phase(key=2, X=[[1.267e-09 59.08], ..., [6.412e-10 56.06]], T=[0, ..., 84.05], dXdT=[[-0 -0], ..., [-1.233e-10 -0.0003919]]}

In [8]:
from cosmoTransitions import transitionFinder as tf
crit_temps=tf.findCriticalTemperatures(phases_dict, m.Vtot)
Num_relevant_trans=0
###DETERMINE IF THERE COULD BE TWO-STEP FOPTs
for elem in crit_temps:
    if elem["trantype"]==1 and abs(elem["low_vev"][0]-elem["high_vev"][0])>10 and abs(elem["low_vev"][1]-elem["high_vev"][1])>10:
        print("Tunneling is relevant from phase " + str(elem["high_phase"])+ " to " + str(elem["low_phase"])  )
        Tmax=elem["Tcrit"]
        Tmin=phases_dict[elem["high_phase"]].T[0]
        print("max temperature is", Tmax)
        print("min temperature is", Tmin)
        Num_relevant_trans+=1
        high_phase_key=elem["high_phase"]
        low_phase_key=elem["low_phase"]
    else:
        continue
   
dict_output= {"Tmin":Tmin, "Tmax":Tmax, "high_phase": high_phase_key,"low_phase": low_phase_key}
X0=m.phases[dict_output["high_phase"]].X[0]
T0=m.phases[dict_output["high_phase"]].T[0]
stable=not np.any(np.linalg.eig(m.d2V(X0,T0))[0]<=0)


Tunneling is relevant from phase 1 to 0
max temperature is 111.23383111088755
min temperature is 84.57747580815298


In [9]:
dict_output

{'Tmin': 84.57747580815298,
 'Tmax': 111.23383111088755,
 'high_phase': 1,
 'low_phase': 0}

In [30]:


def findminT(T):
    """Function to find the minimum temperature at which the high_vev coexists.
    Written in form for optimization"""
    Xmin=m.findMinimum(X0,T)
    dx=np.sum((Xmin-X0)**2)**0.5
    not_stable= np.any(np.linalg.eig(m.d2V(Xmin,T))[0]<=0)
    if not_stable and (dx<.1) == False or T<0:
        return 500
    else:
        return not not_stable and (dx<.1) + T
    
Tmin_opt=optimize.fminbound(findminT,0,T0)


In [31]:
Tmin_opt

6.632464759423544e-06

In [34]:
T=1e-6
Xmin=m.findMinimum(X0,T)
dx=np.sum((Xmin-X0)**2)**0.5
stable= np.any(np.linalg.eig(m.d2V(Xmin,T))[0]<=0)
Xmin,dx,stable

(array([-6.63640500e-10,  5.90838683e+01]), 3.0383306659800304, True)

In [36]:
findminT(1e-4)

0.0001